In [1]:
import pandas as pd
from src.preprocessing import rename_countries
import os
import country_converter as coco

In [2]:
print(os.getcwd())
# Go up one directory to get to the data folder
data_dir = os.path.join(os.path.dirname(os.getcwd()))
os.chdir(data_dir)
print(os.getcwd())

/home/florian/Documents/GitHub/pytradeshifts/scripts
/home/florian/Documents/GitHub/pytradeshifts


In [3]:
hedlund_excel = pd.read_excel(
    "."
    + os.sep
    + "data"
    + os.sep
    + "validation_data_from_Hedlung_2022"
    + os.sep
    + "Ex_15_third percentile_climate impacts.xlsx",
    sheet_name="third percentile",
    engine="openpyxl",
    usecols="A:DI",
    index_col=0,
)

# Sort index and columns
hedlund_excel.sort_index(inplace=True)
hedlund_excel.sort_index(axis=1, inplace=True)

print(hedlund_excel.shape)

# Rename countries using country_converter
hedlund_excel.index = coco.convert(names=hedlund_excel.index, to="name_short")
hedlund_excel.columns = coco.convert(names=hedlund_excel.columns, to="name_short")

hedlund_excel.head()



(112, 112)


,Afghanistan,Albania,Algeria,Argentina,Armenia,Australia,Austria,Azerbaijan,Bangladesh,Belarus,...,Ukraine,United Kingdom,Tanzania,United States,Uruguay,Uzbekistan,Vietnam,Yemen,Zambia,Zimbabwe
Afghanistan,0.000000,0.000000,0.000000e+00,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,...,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.0000
Albania,0.000000,0.000000,0.000000e+00,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,...,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.0000
Algeria,0.000000,0.000000,0.000000e+00,0.0,0.0,0.0,0.00000,0.000000,0.026199,0.0,...,0.000000,1.26650,0.002761,0.055893,0.0000,0.000000,0.007741,0.000000,0.0000,0.0000
Argentina,0.002594,0.010522,1.301639e+06,0.0,0.0,0.0,0.25266,0.001225,246454.724400,0.0,...,0.002269,112.47910,20.432900,423.340100,429.6583,0.007295,217188.820700,6.929800,1.5831,156.6037
Armenia,0.000000,0.000000,1.462500e-01,0.0,0.0,0.0,0.00000,0.000000,0.023679,0.0,...,0.000000,0.01362,0.000000,1.674800,0.0000,0.000000,0.014695,0.025202,0.0000,0.0000


In [4]:
# Load the end results of the R files
hedlund_r = pd.read_csv(
    "."
    + os.sep
    + "data"
    + os.sep
    + "validation_data_from_Hedlung_2022"
    + os.sep
    + "All_Data"
    + os.sep
    + "NEW_Ex_wheat_2018.csv",
    index_col=0,
)
# Rename the countries
# Convert all entries in index and columns to strings
hedlund_r.index = hedlund_r.index.astype(int)
hedlund_r.columns = hedlund_r.columns.astype(int)

# Rename the R names to match the Python names
hedlund_r = rename_countries(
    hedlund_r, "All_Data", "Trade_DetailedTradeMatrix_E", "Area Code"
)
# set diagonal to zero
def pd_fill_diagonal(df_matrix, value=0): 
    mat = df_matrix.values
    n = mat.shape[0]
    mat[range(n), range(n)] = value
    return pd.DataFrame(mat, index=df_matrix.index, columns=df_matrix.columns)

hedlund_r = pd_fill_diagonal(hedlund_r)

# Read in the ISIMIP data, so we can remove those from the R data
ISIMIP = pd.read_csv(
    "."
    + os.sep
    + "data"
    + os.sep
    + "scenario_files"
    + os.sep
    + "ISIMIP_wheat_Hedlung.csv",
    index_col=0,
)

nan_indices = ISIMIP.index[ISIMIP.iloc[:, 0].isnull()].tolist()
# Conver to short names
nan_indices = coco.convert(names=nan_indices, to="name_short") + ["Taiwan"] + ["Macau"]

# Remove the countries without ISIMIP data from the R data
# Take the index of the trade matrix and production data and remove all the countries
# in self.countries_to_remove
countries_to_keep = [
    country
    for country in hedlund_r.index
    if country not in nan_indices
]

hedlund_r = hedlund_r.loc[
    countries_to_keep, countries_to_keep
]

# Sort index and columns
hedlund_r.sort_index(inplace=True)
hedlund_r.sort_index(axis=1, inplace=True)

Replacing country codes with country names in Trade data


In [5]:
assert hedlund_r.shape == hedlund_excel.shape

In [6]:
# Print the countries that are in the excel file but not in the R file
print(
    "Countries in the excel file but not in the R file:",
    set(hedlund_excel.index) - set(hedlund_r.index),
    set(hedlund_excel.columns) - set(hedlund_r.columns),
)

Countries in the excel file but not in the R file: set() {'Greenland'}


In [7]:
# Print the countries that are in the R file but not in the excel file
print(
    "Countries in the R file but not in the excel file:",
    set(hedlund_r.index) - set(hedlund_excel.index),
    set(hedlund_r.columns) - set(hedlund_excel.columns),
)

Countries in the R file but not in the excel file: set() {'Greece'}


In [ ]:
hedlund_r.describe()

,Afghanistan,Albania,Algeria,Argentina,Armenia,Australia,Austria,Azerbaijan,Bangladesh,Belarus,...,Uganda,Ukraine,United Kingdom,United States,Uruguay,Uzbekistan,Vietnam,Yemen,Zambia,Zimbabwe
count,112.000000,112.000000,1.120000e+02,112.000000,112.000000,112.000000,112.000000,112.000000,1.120000e+02,112.000000,...,112.000000,112.000000,112.000000,1.120000e+02,112.000000,1.120000e+02,1.120000e+02,1.120000e+02,112.000000,112.000000
mean,3856.911502,2394.667839,7.519660e+04,0.020280,2430.869772,0.596729,5702.282533,8855.617488,4.320811e+04,2572.895474,...,3398.880001,18.673533,15638.302462,1.647199e+04,3.858855,2.053319e+04,3.856073e+04,2.707685e+04,71.965078,673.644649
std,36454.232798,18450.535613,5.320922e+05,0.190624,25319.158943,4.428766,31770.732279,71489.263669,2.174094e+05,27009.760290,...,24308.107149,82.406222,57819.278416,1.742579e+05,40.591252,2.157591e+05,2.508773e+05,1.483519e+05,633.715697,3839.892453
min,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
50%,0.000000,0.000000,6.905852e-04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,...,0.000000,0.000000,0.001016,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
75%,0.024753,0.845630,1.221573e+02,0.000000,0.001618,0.000000,4.043216,0.047165,3.460960e-01,0.012369,...,3.034075,0.022956,334.107446,7.524925e-01,0.000000,1.118544e-01,3.871708e+00,2.915975e+00,0.579366,1.375791
max,384867.355829,182890.497109,5.412390e+06,2.011911,267974.489171,43.582662,279249.839050,701367.492236,1.856073e+06,285862.324648,...,245203.572221,662.774547,386746.125140,1.844178e+06,429.593091,2.283470e+06,2.462938e+06,1.342162e+06,6696.148902,28249.868322


In [ ]:
hedlund_excel.describe()

,Afghanistan,Albania,Algeria,Argentina,Armenia,Australia,Austria,Azerbaijan,Bangladesh,Belarus,...,Ukraine,United Kingdom,Tanzania,United States,Uruguay,Uzbekistan,Vietnam,Yemen,Zambia,Zimbabwe
count,112.000000,112.000000,1.120000e+02,112.000000,112.000000,112.000000,112.000000,112.000000,1.120000e+02,112.000000,...,112.000000,112.000000,112.000000,1.120000e+02,112.000000,1.120000e+02,1.120000e+02,1.120000e+02,112.000000,112.000000
mean,3856.910700,2394.669624,7.519687e+04,0.019652,2430.863092,0.589161,5706.098211,8855.624992,4.320811e+04,2572.893979,...,18.671170,15638.746778,8324.935304,1.646734e+04,3.861094,2.053320e+04,3.886817e+04,2.715717e+04,71.964271,673.647141
std,36454.253655,18450.564258,5.321423e+05,0.196234,25319.104825,4.421537,31793.057153,71489.309157,2.174046e+05,27009.778327,...,82.400636,57815.924191,72059.671228,1.742074e+05,40.597364,2.157592e+05,2.528775e+05,1.487917e+05,633.711539,3839.816721
min,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
50%,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,...,0.000000,0.008347,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
75%,0.011982,0.854115,1.223233e+02,0.000000,0.001531,0.000000,4.078900,0.045665,2.210875e-01,0.012397,...,0.022640,328.150975,5.876500,7.635600e-01,0.000000,7.468150e-02,3.895225e+00,2.946300e+00,0.577923,1.391450
max,384867.226600,182890.653800,5.412940e+06,2.074000,267973.915900,43.542300,279444.694600,701367.879500,1.856002e+06,285862.512700,...,662.739900,386730.354000,757317.659900,1.843644e+06,429.658300,2.283471e+06,2.482568e+06,1.346143e+06,6696.107200,28248.680600


In [21]:
# Go through all the countries and check if the values are the same
for origin_country in hedlund_excel.index:
    for destination_country in hedlund_excel.columns:
        # skip Greece and Greenland
        if origin_country in ["Greece", "Greenland"] or destination_country in [ "Greece", "Greenland"]:
            continue

        excel_value = hedlund_excel.loc[origin_country, destination_country]
        r_value = hedlund_r.loc[origin_country, destination_country]
        # Print all values with a difference of more than 10
        if abs(excel_value - r_value) > 1000:
            print(
                "Values not the same for",
                origin_country,
                destination_country,
                "excel:",
                excel_value,
                "R:",
                r_value,
            )
      

Values not the same for Argentina Vietnam excel: 217188.8207 R: 215379.925854629
Values not the same for Australia Pakistan excel: 3.4072 R: 1418.91756952867
Values not the same for Australia Vietnam excel: 956682.9386 R: 949117.606085805
Values not the same for Australia Yemen excel: 474836.3531 R: 473432.041158998
Values not the same for Canada Vietnam excel: 255359.6396 R: 253341.776966909
Values not the same for Egypt Libya excel: 1730.1702 R: 4404.94392469277
Values not the same for France Netherlands excel: 2604544.6024 R: 2603125.89125999
Values not the same for Iran Iraq excel: 0.0 R: 1966.58037248061
Values not the same for Iran Oman excel: 68031.7856 R: 39341.6597421758
Values not the same for Russia Cyprus excel: 49333.2354 R: 51568.0617301719
Values not the same for Russia Egypt excel: 9557355.0602 R: 9554030.96036882
Values not the same for Russia Libya excel: 329691.787 R: 333071.779049095
Values not the same for Russia Pakistan excel: 22.8597 R: 4035.80286272242
Values n